# Obtain Evaluation on MIMIC dataset for OASIS and SAPS II

In [ ]:
import mimic_pipeline.utils as utils
user = input('Enter your username: ')
password = input('Enter your password: ')
loader = utils.DataBaseLoader(user=user, password=password)
oasis_df = loader['oasis']
sapsii_df = loader['sapsii']
oasis_df.head()

In [ ]:
sapsii_df.head()

In [ ]:
import pandas as pd
train = pd.read_csv("data/TRAIN-union-features.csv")
test = pd.read_csv("data/TEST-union-features.csv")
all = pd.concat([train, test], axis=0)
print(all.shape)
all.head()

In [ ]:
from sklearn.metrics import auc, precision_recall_curve, roc_curve, brier_score_loss
from sklearn.model_selection import StratifiedKFold
from mimic_pipeline.metric import get_calibration_curve
import numpy as np
import joblib

utils.seed_everything()
oasis_auroc, sapsii_auroc = [], []
oasis_auprc, sapsii_auprc = [], []
oasis_hl, sapsii_hl = [], []
oasis_brier, sapsii_brier = [], []
oasis_smr, sapsii_smr = [], []
for fold in range(1,6):
    test_df = pd.read_csv(f"data/k-fold-id/TEST-union-features-fold{fold}.csv")
    copy = test_df[['subject_id', 'hadm_id', 'icustay_id']].copy()
    test_df = test_df.merge(oasis_df[['subject_id', 'icustay_id', 'hadm_id', 'oasis_prob']], on=['subject_id', 'icustay_id', 'hadm_id'], how='inner')
    test_df = test_df.merge(sapsii_df[['subject_id', 'icustay_id', 'hadm_id', 'sapsii_prob']], on=['subject_id', 'icustay_id', 'hadm_id'], how='inner')
    pd.testing.assert_frame_equal(test_df[['subject_id', 'hadm_id', 'icustay_id']], copy)
    fpr, tpr, _ = roc_curve(test_df['hospital_expire_flag'], test_df['oasis_prob'])
    precision, recall, _ = precision_recall_curve(test_df['hospital_expire_flag'], test_df['oasis_prob'])
    prob_true, prob_pred, h, p = get_calibration_curve(test_df['hospital_expire_flag'], test_df['oasis_prob'])
    oasis_smr.append(np.sum(test_df['hospital_expire_flag'].replace({-1: 0})) / np.sum(test_df['oasis_prob']) )
    oasis_brier.append(brier_score_loss(test_df['hospital_expire_flag'], test_df['oasis_prob']))
    oasis_auroc.append(auc(fpr, tpr))
    oasis_auprc.append(auc(recall, precision))
    oasis_hl.append(h)
    
    fpr, tpr, _ = roc_curve(test_df['hospital_expire_flag'], test_df['sapsii_prob'])
    precision, recall, _ = precision_recall_curve(test_df['hospital_expire_flag'], test_df['sapsii_prob'])
    prob_true, prob_pred, h, p = get_calibration_curve(test_df['hospital_expire_flag'], test_df['sapsii_prob'], strategy='quantile')
    sapsii_brier.append(brier_score_loss(test_df['hospital_expire_flag'], test_df['sapsii_prob']))
    sapsii_smr.append(np.sum(test_df['hospital_expire_flag'].replace({-1: 0})) / np.sum(test_df['sapsii_prob']))
    sapsii_auroc.append(auc(fpr, tpr))
    sapsii_auprc.append(auc(recall, precision))
    sapsii_hl.append(h)

oasis_stats = {'auroc': oasis_auroc, 'auprc': oasis_auprc, 'c-stat': oasis_hl, 'smr': oasis_smr, 'brier': oasis_brier}
sapsii_stats = {'auroc': sapsii_auroc, 'auprc': sapsii_auprc, 'c-stat': sapsii_hl, 'smr': sapsii_smr, 'brier': sapsii_brier}
joblib.dump(oasis_stats, "results/oasis_mimic-stats")
joblib.dump(sapsii_stats, "results/sapsii_mimic-stats")
    
print(f"OASIS AUROC: {np.mean(oasis_auroc):.3f} +/- {np.std(oasis_auroc):.3f}")
print(f"OASIS AUPRC: {np.mean(oasis_auprc):.3f} +/- {np.std(oasis_auprc):.3f}")
print(f"OASIS HL C-stat: {np.mean(oasis_hl):.3f} +/- {np.std(oasis_hl):.3f}")
print(f"OASIS SMR: {np.mean(oasis_smr):.3f} +/- {np.std(oasis_smr):.3f}")
print(f"OASIS Brier: {np.mean(oasis_brier):.3f} +/- {np.std(oasis_brier):.3f}\n")
print(f"SAPSII AUROC: {np.mean(sapsii_auroc):.3f} +/- {np.std(sapsii_auroc):.3f}")
print(f"SAPSII AUPRC: {np.mean(sapsii_auprc):.3f} +/- {np.std(sapsii_auprc):.3f}")
print(f"SAPSII HL C-stat: {np.mean(sapsii_hl):.3f} +/- {np.std(sapsii_hl):.3f}")
print(f"SAPSII SMR: {np.mean(sapsii_smr):.3f} +/- {np.std(sapsii_smr):.3f}")
print(f"SAPSII Brier: {np.mean(sapsii_brier):.3f} +/- {np.std(sapsii_brier):.4f}")

### Disease Specific

In [ ]:
sofa_df = loader['sofa_revised']
sofa_df.head()

In [ ]:
sofa_df['sofa_emp_prob'].unique()

In [ ]:
import joblib
utils.seed_everything()

def evaluate_baseline_disease(disease):
    print(f"Evaluating {disease}...")
    oasis_auroc, sapsii_auroc, sofa_auroc = [], [], []
    oasis_auprc, sapsii_auprc, sofa_auprc = [], [], []
    oasis_hl, sapsii_hl, sofa_hl = [], [], []
    oasis_brier, sapsii_brier, sofa_brier = [], [], []
    oasis_smr, sapsii_smr, sofa_smr = [], [], []
    for fold in range(1,6):
        test_df = pd.read_csv(f"data/mimic-disease/k-fold-id/TEST-{disease}-fold{fold}.csv")
        copy = test_df[['subject_id', 'hadm_id', 'icustay_id']].copy()
        test_df = test_df.merge(oasis_df[['subject_id', 'icustay_id', 'hadm_id', 'oasis_prob']], on=['subject_id', 'icustay_id', 'hadm_id'], how='inner')
        test_df = test_df.merge(sapsii_df[['subject_id', 'icustay_id', 'hadm_id', 'sapsii_prob']], on=['subject_id', 'icustay_id', 'hadm_id'], how='inner')
        test_df = test_df.merge(sofa_df[['subject_id', 'icustay_id', 'hadm_id', 'sofa_emp_prob']], on=['subject_id', 'icustay_id', 'hadm_id'], how='inner')
        pd.testing.assert_frame_equal(test_df[['subject_id', 'hadm_id', 'icustay_id']], copy)
        
        fpr, tpr, _ = roc_curve(test_df['hospital_expire_flag'], test_df['oasis_prob'])
        precision, recall, _ = precision_recall_curve(test_df['hospital_expire_flag'], test_df['oasis_prob'])
        prob_true, prob_pred, h, p = get_calibration_curve(test_df['hospital_expire_flag'], test_df['oasis_prob'])
        oasis_smr.append(np.sum(test_df['hospital_expire_flag'].replace({-1: 0})) / np.sum(test_df['oasis_prob']) )
        oasis_brier.append(brier_score_loss(test_df['hospital_expire_flag'], test_df['oasis_prob']))
        oasis_auroc.append(auc(fpr, tpr))
        oasis_auprc.append(auc(recall, precision))
        oasis_hl.append(h)
        
        fpr, tpr, _ = roc_curve(test_df['hospital_expire_flag'], test_df['sapsii_prob'])
        precision, recall, _ = precision_recall_curve(test_df['hospital_expire_flag'], test_df['sapsii_prob'])
        prob_true, prob_pred, h, p = get_calibration_curve(test_df['hospital_expire_flag'], test_df['sapsii_prob'], strategy='quantile')
        sapsii_brier.append(brier_score_loss(test_df['hospital_expire_flag'], test_df['sapsii_prob']))
        sapsii_smr.append(np.sum(test_df['hospital_expire_flag'].replace({-1: 0})) / np.sum(test_df['sapsii_prob']))
        sapsii_auroc.append(auc(fpr, tpr))
        sapsii_auprc.append(auc(recall, precision))
        sapsii_hl.append(h)
        
        fpr, tpr, _ = roc_curve(test_df['hospital_expire_flag'], test_df['sofa_emp_prob'])
        precision, recall, _ = precision_recall_curve(test_df['hospital_expire_flag'], test_df['sofa_emp_prob'])
        sofa_auroc.append(auc(fpr, tpr))
        sofa_auprc.append(auc(recall, precision))
    
    sofa_stats = {'auroc': sofa_auroc, 'auprc': sofa_auprc}
    sapsii_stats = {'auroc': sapsii_auroc, "auprc": sapsii_auprc}
    oasis_stats = {'auroc': oasis_auroc, "auprc": oasis_auprc}
    
    joblib.dump(sofa_stats, f"results/disease/{disease}/sofa_stats")
    joblib.dump(sapsii_stats, f"results/disease/{disease}/sapsii_stats")
    joblib.dump(oasis_stats, f"results/disease/{disease}/oasis_stats")
        
    print(f"OASIS AUROC: {np.mean(oasis_auroc):.3f} +/- {np.std(oasis_auroc):.3f}")
    print(f"OASIS AUPRC: {np.mean(oasis_auprc):.3f} +/- {np.std(oasis_auprc):.3f}")
    print(f"OASIS HL C-stat: {np.mean(oasis_hl):.3f} +/- {np.std(oasis_hl):.3f}")
    print(f"OASIS SMR: {np.mean(oasis_smr):.3f} +/- {np.std(oasis_smr):.3f}")
    print(f"OASIS Brier: {np.mean(oasis_brier):.3f} +/- {np.std(oasis_brier):.3f}\n")
    
    print(f"SAPSII AUROC: {np.mean(sapsii_auroc):.3f} +/- {np.std(sapsii_auroc):.3f}")
    print(f"SAPSII AUPRC: {np.mean(sapsii_auprc):.3f} +/- {np.std(sapsii_auprc):.3f}")
    print(f"SAPSII HL C-stat: {np.mean(sapsii_hl):.3f} +/- {np.std(sapsii_hl):.3f}")
    print(f"SAPSII SMR: {np.mean(sapsii_smr):.3f} +/- {np.std(sapsii_smr):.3f}")
    print(f"SAPSII Brier: {np.mean(sapsii_brier):.3f} +/- {np.std(sapsii_brier):.4f}\n")
    
    print(f"SOFA AUROC: {np.mean(sofa_auroc):.3f} +/- {np.std(sofa_auroc):.3f}")
    print(f"SOFA AUPRC: {np.mean(sofa_auprc):.3f} +/- {np.std(sofa_auprc):.3f}")

evaluate_baseline_disease('sepsis')

In [ ]:
evaluate_baseline_disease('akf')

In [ ]:
evaluate_baseline_disease('ami')

In [ ]:
evaluate_baseline_disease('heart_failure')